In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 204.7MB 66kB/s 
     |████████████████████████████████| 204kB 42.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044182 sha256=a358d04dfd69eaad7e8bea2f146a97da4fa97be43eec8d758d09c3a93e0ac119
  Stored in directory: /root/.cache/pip/wheels/57/27/4d/ddacf7143f8d5b76c45c61ee2e43d9f8492fc5a8e78ebd7d37
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 33 not upgraded.
Need to get 35.8 MB of archives.
After this 

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [3]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

# Organización de Datos 75.06. Primer Cuatrimestre de 2017. Examen parcial, primera oportunidad:


### 1) Se tiene información estadística de la temporada regular de todos los jugadores de la NBA en un RDD de tuplas con el siguiente formato: (id_jugador, nombre, promedio_puntos, promedio_asistencias, promedio_robos, promedio_bloqueos, promedio_rebotes, promedio_faltas). Un analista de la cadena ESPN está trabajando con un RDD que corresponde a la primera ronda de playoffs y que tiene el siguiente formato: (id_jugador, id_partido, timestamp, cantidad_puntos, cantidad_rebotes, cantidad_bloqueos, cantidad_robos, cantidad_asistencias, cantidad_faltas). En base a estos RDDs se quiere programar en PySpark un programa que genere un RDD con los nombres (sin duplicados) de los jugadores que lograron en algún partido de playoffs una cantidad de asistencias mayor a su promedio histórico. (****) (15 pts)

In [11]:
# (id_jugador, nombre, promedio_puntos, promedio_asistencias, promedio_robos, promedio_bloqueos, promedio_rebotes, promedio_faltas).
historico = [(1010, 'Jack S', 20, 32, 17, 11, 10, 3),
             (1011, 'George', 30, 33, 19, 20, 11, 13),
             (1012, 'John F', 20, 33, 27, 21, 12, 31),
             (1013, 'Matt T', 11, 30, 37, 12, 13, 31),
             (1014, 'Lyon R', 16, 59, 22, 19, 19, 30),
             (1015, 'Floyd ', 29, 42, 45, 16, 18, 4),
             (1016, 'Hether', 35, 43, 36, 14, 17, 10),
             (1017, 'Jessez', 44, 34, 33, 12, 17, 6),
             (1018, 'Brians', 24, 39, 11, 13, 16, 7),
             (1019, 'Rambos', 20, 11, 9, 14, 11, 8),
             (1020, 'Wick Q', 24, 14, 7, 21, 21, 11),
             (1021, 'Finger', 25, 15, 37, 22, 22, 19),
             (1022, 'Leenyr', 40, 15, 22, 29, 21, 14),
             (1023, 'Chad V', 30, 31, 18, 21, 22, 13),
             (1024, 'Tony W', 16, 30, 29, 11, 21, 21)]
# (id_jugador, id_partido, timestamp, cantidad_puntos, cantidad_rebotes, cantidad_bloqueos, cantidad_robos, cantidad_asistencias, cantidad_faltas)
playoffs = [(1010, 1, 2019-10-11, 10, 9, 24, 11, 19, 21 ),
            (1012, 1, 2019-10-11, 11, 19, 22, 1, 29, 0 ),
            (1013, 1, 2019-10-11, 21, 29, 21, 10, 22, 20 ),
            (1015, 2, 2019-9-11, 40, 19, 20, 11, 12, 3 ),
            (1018, 2, 2019-9-11, 31, 9, 34, 21, 13, 4 ),
            (1019, 2, 2019-9-11, 28, 14, 11, 22, 14, 45 ),
            (1020, 3, 2019-11-11, 22, 11, 14, 1, 24, 17 ),
            (1023, 3, 2019-11-11, 9, 19, 18, 9, 26, 3 ),
            (1024, 3, 2019-11-11, 7, 41, 14, 8, 28, 2 )]

In [12]:
historicoRDD = sc.parallelize(historico)
playoffsRDD = sc.parallelize(playoffs)

In [13]:
historicoRDD.collect()

[(1010, 'Jack S', 20, 32, 17, 11, 10, 3),
 (1011, 'George', 30, 33, 19, 20, 11, 13),
 (1012, 'John F', 20, 33, 27, 21, 12, 31),
 (1013, 'Matt T', 11, 30, 37, 12, 13, 31),
 (1014, 'Lyon R', 16, 59, 22, 19, 19, 30),
 (1015, 'Floyd ', 29, 42, 45, 16, 18, 4),
 (1016, 'Hether', 35, 43, 36, 14, 17, 10),
 (1017, 'Jessez', 44, 34, 33, 12, 17, 6),
 (1018, 'Brians', 24, 39, 11, 13, 16, 7),
 (1019, 'Rambos', 20, 11, 9, 14, 11, 8),
 (1020, 'Wick Q', 24, 14, 7, 21, 21, 11),
 (1021, 'Finger', 25, 15, 37, 22, 22, 19),
 (1022, 'Leenyr', 40, 15, 22, 29, 21, 14),
 (1023, 'Chad V', 30, 31, 18, 21, 22, 13),
 (1024, 'Tony W', 16, 30, 29, 11, 21, 21)]

In [14]:
playoffsRDD.collect()

[(1010, 1, 1998, 10, 9, 24, 11, 19, 21),
 (1012, 1, 1998, 11, 19, 22, 1, 29, 0),
 (1013, 1, 1998, 21, 29, 21, 10, 22, 20),
 (1015, 2, 1999, 40, 19, 20, 11, 12, 3),
 (1018, 2, 1999, 31, 9, 34, 21, 13, 4),
 (1019, 2, 1999, 28, 14, 11, 22, 14, 45),
 (1020, 3, 1997, 22, 11, 14, 1, 24, 17),
 (1023, 3, 1997, 9, 19, 18, 9, 26, 3),
 (1024, 3, 1997, 7, 41, 14, 8, 28, 2)]

In [17]:
playoffs_short = playoffsRDD.map(lambda x: (x[0],x[7]))
playoffs_short.collect()

[(1010, 19),
 (1012, 29),
 (1013, 22),
 (1015, 12),
 (1018, 13),
 (1019, 14),
 (1020, 24),
 (1023, 26),
 (1024, 28)]

In [20]:
historico_short = historicoRDD.map(lambda x: (x[0],(x[1],x[3])))
historico_short.collect()

[(1010, ('Jack S', 32)),
 (1011, ('George', 33)),
 (1012, ('John F', 33)),
 (1013, ('Matt T', 30)),
 (1014, ('Lyon R', 59)),
 (1015, ('Floyd ', 42)),
 (1016, ('Hether', 43)),
 (1017, ('Jessez', 34)),
 (1018, ('Brians', 39)),
 (1019, ('Rambos', 11)),
 (1020, ('Wick Q', 14)),
 (1021, ('Finger', 15)),
 (1022, ('Leenyr', 15)),
 (1023, ('Chad V', 31)),
 (1024, ('Tony W', 30))]

In [24]:
historico_vs_playoffs = historico_short.join(playoffs_short)
historico_vs_playoffs.collect()

[(1012, (('John F', 33), 29)),
 (1020, (('Wick Q', 14), 24)),
 (1024, (('Tony W', 30), 28)),
 (1013, (('Matt T', 30), 22)),
 (1010, (('Jack S', 32), 19)),
 (1018, (('Brians', 39), 13)),
 (1015, (('Floyd ', 42), 12)),
 (1019, (('Rambos', 11), 14)),
 (1023, (('Chad V', 31), 26))]

In [28]:
comparativa_asistencias = historico_vs_playoffs.map(lambda x: (x[1][0][0],(x[1][0][1],x[1][1])))
comparativa_asistencias.collect()

[('John F', (33, 29)),
 ('Wick Q', (14, 24)),
 ('Tony W', (30, 28)),
 ('Matt T', (30, 22)),
 ('Jack S', (32, 19)),
 ('Brians', (39, 13)),
 ('Floyd ', (42, 12)),
 ('Rambos', (11, 14)),
 ('Chad V', (31, 26))]

In [30]:
comparativa_asistencias_supera_promedio = comparativa_asistencias.filter(lambda x: x[1][1]>x[1][0])
comparativa_asistencias_supera_promedio.collect()

[('Wick Q', (14, 24)), ('Rambos', (11, 14))]